In [3]:
import os
os.environ["CHECKPOINTS_PATH"] = "../checkpoints"

import shutil
from typing import Any

import numpy as np
import pandas as pd
from src.api.services import embeddings_service
from tqdm import tqdm

from experiment.settings import (
    FULLY_LABELED_RECORDINGS,
    GAZE_SEGMENTATION_RESULTS_PATH,
    OBJECT_DATASETS_PATH,
    TRIAL_RECORDING_IDS,
)

In [4]:
dinov2 = embeddings_service.load_model()

In [5]:
file = np.load(
    GAZE_SEGMENTATION_RESULTS_PATH / "32f02db7-adc0-4556-a2da-ed2ba60a58c9" / "4.npz"
)
file.files

['boxes',
 'rois',
 'masks',
 'object_ids',
 'frame_idx',
 'gaze_position',
 'confidences']

In [6]:
def create_objects_dataframe(gaze_segmentation_results: list[Any]):
    object_results = []
    for result in gaze_segmentation_results:
        # get the first batch of embeddings since we only have one batch
        embeddings, _, _ = list(
            embeddings_service.get_embeddings(dinov2, result["rois"])
        )[0]
        boxes = result["boxes"]

        for i in range(len(result["rois"])):
            x1, y1, x2, y2 = boxes[i]
            object_results.append({
                "frame_idx": result["frame_idx"],
                "object_id": result["object_ids"][i],
                "confidence": result["confidences"][i],
                "embedding": embeddings[i].tolist(),
                "mask_area": np.sum(result["masks"][i]),
                "x1": x1,
                "y1": y1,
                "x2": x2,
                "y2": y2,
            })

    return pd.DataFrame(object_results)

In [7]:
if OBJECT_DATASETS_PATH.exists():
    shutil.rmtree(OBJECT_DATASETS_PATH)
OBJECT_DATASETS_PATH.mkdir(parents=True, exist_ok=True)

for trial_recording_id in tqdm(FULLY_LABELED_RECORDINGS, desc="Processing trial recordings"):
    # Load gaze segmentation results for this recording
    gaze_segmentation_results_path = GAZE_SEGMENTATION_RESULTS_PATH / trial_recording_id
    gaze_segmentation_results = list(gaze_segmentation_results_path.iterdir())
    gaze_segmentation_results.sort(key=lambda x: int(x.stem))
    gaze_segmentation_results = [
        np.load(result, allow_pickle=True) for result in gaze_segmentation_results
    ]

    objects_df = create_objects_dataframe(gaze_segmentation_results)
    objects_df.to_csv(OBJECT_DATASETS_PATH / f"{trial_recording_id}.csv", index=False)

Processing trial recordings: 100%|██████████| 14/14 [02:08<00:00,  9.20s/it]
